In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 읽기
img = cv2.imread("img/paper.jpg")

# 그레이 스케일로 변환을 합니다. -> 흑백 이미지로 바꾸는 이유는 findContour가 흑백이미지 또는 이진화된 이미지만 적용할 수 있기 때문이다.
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 적응형 스레스홀드로 흑백 이미지를 이진화 이미지로 바꿔줍니다.
th = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 3)

# 가우시안 블러로 노이즈를 제거합니다.
blur = cv2.GaussianBlur(th, (5, 5), 0)

# 케니 엣지로 경계를 검출합니다.
edges = cv2.Canny(blur, 50, 260)

# 모든 컨투어들을 찾아냅니다.
_, contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL,  cv2.CHAIN_APPROX_SIMPLE) # 모든 계층 정보를 트리 구조로 제공/컨투어 꼭지점 좌표만 이용

# (확인) 컨투어 갯수와 계층 트리를 출력합니다.
print("number of contours: ", len(contours))
print("hierarchy: ", hierarchy)

# (hint) 컨투어들 중에 영역 크기 순으로 정렬합니다.
contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

# approxPolyDP 함수를 사용하여 꼭지점의 좌표를 추출합니다. 
approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

# approx의 사이즈를 다시 조정합니다.
approx = np.array(approx)
resized_approx = np.resize(approx, (4,2))
print('resized approx: ', resized_approx)

# 원본 사진의 크기를 전역변수 rows, cols에 담아줍니다.
rows, cols = img.shape[:2]

# 원근 변환을 해줍니다.
pts1 = np.float32([resized_approx[0], resized_approx[1], resized_approx[2], resized_approx[3]])
pts2 = np.float32([[0,0], [0, rows], [cols, rows], [cols, 0]])

mtrx = cv2.getPerspectiveTransform(pts1, pts2)

dst = cv2.warpPerspective(img, mtrx, (cols, rows))

# 새 창으로 결과를 출력합니다.
cv2.imshow('thresh', th)
cv2.imshow('blur', blur)
cv2.imshow('edges', edges)
cv2.imshow('result', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

number of contours:  683
hierarchy:  [[[  1  -1  -1  -1]
  [  2   0  -1  -1]
  [  3   1  -1  -1]
  ...
  [681 679  -1  -1]
  [682 680  -1  -1]
  [ -1 681  -1  -1]]]
resized approx:  [[101  54]
 [ 23 706]
 [569 695]
 [497 103]]


In [ ]:
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

# img = cv2.imread("img/paper.jpg")
# draw = img.copy()

# # 그레이스 스케일 변환 및 케니 엣지
# imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# th = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 3)
# blur = cv2.GaussianBlur(th, (5, 5), 0)
# edges = cv2.Canny(blur, 50, 260)

# # 컨투어 찾기
# _, cnts, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL,  cv2.CHAIN_APPROX_SIMPLE)

# # 모든 컨투어 그리기



# # 컨투어들 중에 영역 크기 순으로 정렬
# cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
# for c in cnts:
#     # 영역이 가장 큰 컨투어 부터 근사 컨투어 단순화
#     peri = cv2.arcLength(c, True)   # 둘레 길이
#     # 둘레 길이의 0.02 근사값으로 근사화
#     vertices = cv2.approxPolyDP(c, 0.02 * peri, True) 
#     if len(vertices) == 4: # 근사한 꼭지점이 4개면 중지
#         break
# pts = vertices.reshape(4, 2) # N x 1 x 2 배열을 4 x 2크기로 조정
# for x,y in pts:
#     cv2.circle(draw, (x,y), 10, (0,255,0), -1) # 좌표에 초록색 동그라미 표시
# merged = np.hstack((img, draw))

# # 좌표 4개 중 상하좌우 찾기 ---② 
# sm = pts.sum(axis=1)                 # 4쌍의 좌표 각각 x+y 계산
# diff = np.diff(pts, axis = 1)       # 4쌍의 좌표 각각 x-y 계산

# topLeft = pts[np.argmin(sm)]         # x+y가 가장 작은 값이 좌상단 좌표
# bottomRight = pts[np.argmax(sm)]     # x+y가 가장 큰 값이 좌상단 좌표
# topRight = pts[np.argmin(diff)]     # x-y가 가장 작은 값이 우상단 좌표
# bottomLeft = pts[np.argmax(diff)]   # x-y가 가장 큰 값이 좌하단 좌표

# # 변환 전 4개 좌표 
# pts1 = np.float32([topLeft, topRight, bottomRight , bottomLeft])

# # 변환 후 영상에 사용할 서류의 폭과 높이 계산 ---③ 
# w1 = abs(bottomRight[0] - bottomLeft[0])    # 상단 좌우 좌표간의 거리
# w2 =           # 하당 좌우 좌표간의 거리
# h1 =       # 우측 상하 좌표간의 거리
# h2 =         # 좌측 상하 좌표간의 거리
# width =                       # 두 좌우 거리간의 최대값이 서류의 폭
# height =                      # 두 상하 거리간의 최대값이 서류의 높이

# # 변환 후 4개 좌표
# pts2 = 

# # 변환 행렬 계산 
# mtrx = 

# # 원근 변환 적용
# result = 

# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.show()
# plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))
# plt.show()
# plt.imshow(cv2.cvtColor(draw, cv2.COLOR_BGR2RGB))
# plt.show()
# plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
# plt.show()